<a href="https://colab.research.google.com/github/daniel-gomm/kd-documents/blob/IMRaD/Unpaywall_IMRaD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers



     |████████████████████████████████| 2.6 MB 7.3 MB/s 
     |████████████████████████████████| 3.3 MB 48.9 MB/s 
     |████████████████████████████████| 636 kB 36.1 MB/s 
     |████████████████████████████████| 895 kB 63.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import torch
import pandas as pd
from tqdm.notebook import tqdm

from sklearn.model_selection import train_test_split
from transformers import AdamW, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

from transformers import BertTokenizer
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

from transformers import BertForSequenceClassification
from transformers import BertModel, BertConfig
import random
import numpy as np
import operator

from transformers import AutoTokenizer, AutoModel


In [ ]:
import json
import glob
import os
import csv
from nltk.tokenize import sent_tokenize
import nltk
import bs4
from bs4 import BeautifulSoup

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

Declaring the different folders for the input and csv for the section output.

In [ ]:
folder_path_tei="/content/drive/MyDrive/Seminar_KD_aifb/Unpaywall/unpaywall_Transfer"
folder_path_tei_test="/content/drive/MyDrive/Seminar_KD_aifb/Unpaywall/Test"
global unpaywallOutput
global TestUnpaywallOutput

unpaywallOutput="/content/drive/MyDrive/Seminar_KD_aifb/csv/UnpaywallCSV/CSVUnpaywall.csv"
TestUnpaywallOutput="/content/drive/MyDrive/Seminar_KD_aifb/csv/UnpaywallCSV/TestCSVUnpaywall.csv"
unpaywall="/content/drive/MyDrive/Seminar_KD_aifb/csv/UnpaywallCSV/CSVUnpaywallEdited - CSVUnpaywall.csv"
unpaywallTesting="/content/drive/MyDrive/Seminar_KD_aifb/csv/UnpaywallCSV/TestCSVUnpaywallEdited - TestCSVUnpaywall.csv"

unarxiveTestingInput="/content/drive/MyDrive/Seminar_KD_aifb/csv/jsonOutput.csv"
unarxiveTesting="/content/drive/MyDrive/Seminar_KD_aifb/csv/RelatedBlock/relatedBlockEdited - relatedBlock.csv"

unpaywallBig="/content/drive/MyDrive/Seminar_KD_aifb/Unpaywall/Unpaywall_Data_Random/unpaywall_Transfer"
unpaywallBigcsv="/content/drive/MyDrive/Seminar_KD_aifb/csv/UnpaywallCSV/UnpaywallBig.csv"

csvIntro="/content/drive/MyDrive/Seminar_KD_aifb/csv/UnpaywallCSV/UnpaywallSections/introcsv.csv"
csvRelated="/content/drive/MyDrive/Seminar_KD_aifb/csv/UnpaywallCSV/UnpaywallSections/relatedcsv.csv"
csvMethod="/content/drive/MyDrive/Seminar_KD_aifb/csv/UnpaywallCSV/UnpaywallSections/methodcsv.csv"
csvResults="/content/drive/MyDrive/Seminar_KD_aifb/csv/UnpaywallCSV/UnpaywallSections/resultscsv.csv"
csvDiscussion="/content/drive/MyDrive/Seminar_KD_aifb/csv/UnpaywallCSV/UnpaywallSections/discussioncsv.csv"


editedIntro="/content/drive/MyDrive/Seminar_KD_aifb/csv/UnpaywallCSV/UnpaywallSectionsEdited/introcsv - introcsv.csv"
editedRelated="/content/drive/MyDrive/Seminar_KD_aifb/csv/UnpaywallCSV/UnpaywallSectionsEdited/relatedcsv - relatedcsv.csv"
editedMethods="/content/drive/MyDrive/Seminar_KD_aifb/csv/UnpaywallCSV/UnpaywallSectionsEdited/methodcsv - methodcsv.csv"
editedResults="/content/drive/MyDrive/Seminar_KD_aifb/csv/UnpaywallCSV/UnpaywallSectionsEdited/resultscsv - resultscsv.csv"
editedDiscussion="/content/drive/MyDrive/Seminar_KD_aifb/csv/UnpaywallCSV/UnpaywallSectionsEdited/discussioncsv - discussioncsv.csv"

In [ ]:
editedTestIntro="/content/drive/MyDrive/Seminar_KD_aifb/csv/UnpaywallCSV/UnpaywallEditedTestingSections/introtesting - introtesting.csv"
editedTestRelated="/content/drive/MyDrive/Seminar_KD_aifb/csv/UnpaywallCSV/UnpaywallEditedTestingSections/relatedTesting - relatedTesting.csv"
editedTestMethods="/content/drive/MyDrive/Seminar_KD_aifb/csv/UnpaywallCSV/UnpaywallEditedTestingSections/methodsTesting - methodsTesting.csv"
editedTestResults="/content/drive/MyDrive/Seminar_KD_aifb/csv/UnpaywallCSV/UnpaywallEditedTestingSections/resultsTesting - resultsTesting (1).csv"
editedTestDiscussion="/content/drive/MyDrive/Seminar_KD_aifb/csv/UnpaywallCSV/UnpaywallSectionsEdited/discussioncsv - discussioncsv.csv"

Creating Test Data

In [ ]:
csvIntro=introTest="/content/drive/MyDrive/Seminar_KD_aifb/csv/UnpaywallCSV/UnpaywallTestingSections/introtesting.csv"
csvRelated=relatedTest="/content/drive/MyDrive/Seminar_KD_aifb/csv/UnpaywallCSV/UnpaywallTestingSections/relatedTesting.csv"
csvMethod=methodTest="/content/drive/MyDrive/Seminar_KD_aifb/csv/UnpaywallCSV/UnpaywallTestingSections/methodsTesting.csv"
csvResults=resultTest="/content/drive/MyDrive/Seminar_KD_aifb/csv/UnpaywallCSV/UnpaywallTestingSections/resultsTesting.csv"
csvDiscussion=discussionTest="/content/drive/MyDrive/Seminar_KD_aifb/csv/UnpaywallCSV/UnpaywallTestingSections/DiscussionTesting.csv"
unpaywallBig=unpaywalltestingFinal="/content/drive/MyDrive/Seminar_KD_aifb/Unpaywall/unpaywall_Testing_Data"

**Methods:**

`toSoup` reads an xml file and turns it into a BeautifulSoup

`findSection` takes a soup and an array of strings. For each div (similar to section) in the soup, the text is extracted and saved in the `divs_text` array.
If the headers of a div contain this section, the whole div is returned. 
If any of the `string` elements are in the header of a section, it is assumed this section belongs to that type. The complete div is returned. It is assumed that each section only appears once in each document, so returning one div should be sufficient.**Text fett markieren**



In [ ]:
"""def toSoup(doc_path):
  with open(doc_path, 'r') as tei:
    soup = BeautifulSoup(tei, 'xml')
    return soup

def findSection(soup,string):
  alldivs=""
  for div in soup.body.find_all("div"):
    divs_text=[]
    rd="Results and Discussion"
    for child in div.children:
      if (not child.name == "listbibl"):
        if (isinstance(child, bs4.element.NavigableString)):
          divs_text.append(str(child))
        elif (isinstance(child, bs4.element.Tag)):
          divs_text.append(child.text)

    if (len(divs_text)>0):
      if (not rd.lower() in divs_text[0].lower()):
        for s in string:
          if (s.lower() in divs_text[0].lower()):
            for d in divs_text:
              alldivs=alldivs+d+" "
            return alldivs
  return"""

'def toSoup(doc_path):\n  with open(doc_path, \'r\') as tei:\n    soup = BeautifulSoup(tei, \'xml\')\n    return soup\n\ndef findSection(soup,string):\n  alldivs=""\n  for div in soup.body.find_all("div"):\n    divs_text=[]\n    rd="Results and Discussion"\n    for child in div.children:\n      if (not child.name == "listbibl"):\n        if (isinstance(child, bs4.element.NavigableString)):\n          divs_text.append(str(child))\n        elif (isinstance(child, bs4.element.Tag)):\n          divs_text.append(child.text)\n\n    if (len(divs_text)>0):\n      if (not rd.lower() in divs_text[0].lower()):\n        for s in string:\n          if (s.lower() in divs_text[0].lower()):\n            for d in divs_text:\n              alldivs=alldivs+d+" "\n            return alldivs\n  return'

**Different sections:**: 


*   `intro` with header `"Introduction"`
*   `method` with header 
*   `related`
*   `discussion`
*   `result`

In [ ]:
#for each paper: 
  #check if there are enough sections (amount wanted specified by user)
    #if there are -->break
  #for each div section in the document: 
    #check if there are any strings from the list with words to search for in the header 
      #if there are: return the div as a parsed version
  #add the divs to an array or directly write them to the csv file (depends on the speed, needs to be tested)


In [ ]:
def findSectionsAndWriteToCsv(inputFolder, outputCsv, maxSectionsToFind, listOfWords, labelOfSection):
  f = open(outputCsv, 'w')
  storageArray=[]
  
  with f:
    writer = csv.writer(f)
    writer.writerow(["Sentence","Section"])

  counter1=0
  for filename in tqdm(glob.glob(os.path.join(inputFolder, '*.xml'))):
    if (counter1>=maxSectionsToFind):
      print(f"{maxSectionsToFind} sections have been found.")
      break

    soup=toSoup(filename)
    
    if (findSection(soup, listOfWords)!=None):
      storageArray.append(findSection(soup, listOfWords))
      counter1=counter1+1

  f = open(outputCsv, 'a')
  with f:
    writer = csv.writer(f)
    for storage in storageArray:
      writer.writerow([storage,labelOfSection])
  print(counter1)



In [ ]:
intro=["Introduction"]
findSectionsAndWriteToCsv(unpaywallBig, editedTestIntro,300,intro,"intro")

related=["Related Work"]
#findSectionsAndWriteToCsv(unpaywallBig, csvRelated,300,related,"related")

method=["Methods and materials", "Materials and methods","Methodology"]
#findSectionsAndWriteToCsv(unpaywallBig, csvMethod,300,method,"method")

results=["Results"]
#findSectionsAndWriteToCsv(unpaywallBig, csvResults,400,results,"results")

discussion=["Discussion","Conclusion"]
#findSectionsAndWriteToCsv(unpaywallBig, csvDiscussion, 400, discussion,"discussion")

  0%|          | 0/2000 [00:00<?, ?it/s]

NameError: ignored

In [ ]:
intro=["Introduction"]
#findSectionsAndWriteToCsv(unpaywallBig, editedTestIntro,300,intro,"intro")

related=["Related Work"]
#findSectionsAndWriteToCsv(unpaywallBig, csvRelated,300,related,"related")

method=["Methods and materials", "Materials and methods","Methodology"]
#findSectionsAndWriteToCsv(unpaywallBig, csvMethod,300,method,"method")

results=["Results"]
#findSectionsAndWriteToCsv(unpaywallBig, csvResults,400,results,"results")

discussion=["Discussion","Conclusion"]
#findSectionsAndWriteToCsv(unpaywallBig, csvDiscussion, 400, discussion,"discussion")

In [ ]:
listWithEdits=[editedIntro,editedRelated,editedMethods,editedResults,editedDiscussion]
listWithLabels=["intro","related","method","results","discussion"]
sectionList=[intro,related,method,results,discussion]
pathHelper="/content/drive/MyDrive/Seminar_KD_aifb/csv/help.csv"
length=100
listIndex=0
counter3=0

f = open(pathHelper, 'w')
with f:
  writer = csv.writer(f)
  writer.writerow(["Sentence","Section"])

for l in listWithEdits:
  dfHelp = pd.read_csv(l)
  dfHelp=dfHelp['Sentence']
  counter=0
  counterRows=0
  #print((dfHelp.shape[0]/2))

  for dfh in dfHelp:
    helper=[]
    sent=sent_tokenize(dfh)
    h=""
    g=""
    if (listIndex == 2):
      if (counterRows<(dfHelp.shape[0]/2)):
        counterRows=counterRows+1
        for string in sectionList[listIndex]:
          if string.lower() in sent[0].lower():
            sent[0]=sent[0].replace(string,"")
            sent[0]=sent[0].replace(string.lower(),"")
            sent[0]=sent[0].replace(string.upper(),"")

            counter3=counter3+1


    for j in range(0,len(sent)):
      g=h+sent[j]
      if (len(g.split())<length):
        h=h+sent[j]
      else:
        counter=counter+1
        helper.append(h)
        h=sent[j]
    f = open(pathHelper, 'a')
    with f:
      writer = csv.writer(f)    
      for k in helper:
        writer.writerow([k,listWithLabels[listIndex]])
  print(f"{counter}" )
  listIndex=listIndex+1
#print(counter3)

NameError: ignored

In [ ]:
listWithEdits=[editedTestIntro,editedTestRelated,editedTestMethods,editedTestResults,editedTestDiscussion]
listWithLabels=["intro","related","method","results","discussion"]
sectionList=[intro,related,method,results,discussion]

pathHelper2="/content/drive/MyDrive/Seminar_KD_aifb/csv/help2.csv"
length=100
listIndex=0


f = open(pathHelper2, 'w')
with f:
  writer = csv.writer(f)
  writer.writerow(["Sentence","Section"])

for l in listWithEdits:
  dfHelp = pd.read_csv(l)
  dfHelp=dfHelp['Sentence']
  counter=0
  for dfh in dfHelp:
    helper=[]
    sent=sent_tokenize(dfh)
    h=""
    g=""
    """for string in sectionList[listIndex]:
      if string.lower() in sent[0].lower():
        sent[0]=sent[0].replace(string,"")
        sent[0]=sent[0].replace(string.lower(),"")
        sent[0]=sent[0].replace(string.upper(),"")"""
    for j in range(0,len(sent)):
      g=h+sent[j]
      if (len(g.split())<length):
        h=h+sent[j]
      else:
        counter=counter+1
        helper.append(h)
        h=sent[j]
    f = open(pathHelper2, 'a')
    with f:
      writer = csv.writer(f)    
      for k in helper:
        writer.writerow([k,listWithLabels[listIndex]])
  print(f"{counter}" )
  listIndex=listIndex+1

1504
755
336
901
2075


Adding ID

In [ ]:
listWithEdits=[editedTestIntro,editedTestRelated,editedTestMethods,editedTestResults,editedTestDiscussion]
listWithLabels=["intro","related","method","results","discussion"]
sectionList=[intro,related,method,results,discussion]
x
pathHelper2="/content/drive/MyDrive/Seminar_KD_aifb/csv/help3.csv"
length=100
listIndex=0
ID=0

f = open(pathHelper2, 'w')
with f:
  writer = csv.writer(f)
  writer.writerow(["Sentence","Section","ID"])

for l in listWithEdits:
  dfHelp = pd.read_csv(l)
  dfHelp=dfHelp['Sentence']
  counter=0
  
  for dfh in dfHelp:
    ID=ID+1
    helper=[]
    sent=sent_tokenize(dfh)
    h=""
    g=""
    """for string in sectionList[listIndex]:
      if string.lower() in sent[0].lower():
        sent[0]=sent[0].replace(string,"")
        sent[0]=sent[0].replace(string.lower(),"")
        sent[0]=sent[0].replace(string.upper(),"")"""
    for j in range(0,len(sent)):
      g=h+sent[j]
      if (len(g.split())<length):
        h=h+sent[j]
      else:
        counter=counter+1
        helper.append(h)
        h=sent[j]
    f = open(pathHelper2, 'a')
    with f:
      writer = csv.writer(f)    
      for k in helper:
        if (type(k) == str):
          if (len(k.split())<150):
            writer.writerow([str(k),listWithLabels[listIndex],ID])
        if (type(k) != str):
          print("lol")

  print(f"{counter}" )
  listIndex=listIndex+1

1504
755
336
901
2075


In [ ]:
pathHelper="/content/drive/MyDrive/Seminar_KD_aifb/csv/help.csv"
df = pd.read_csv(pathHelper)
#df.astype({"Sentence": str})
#testSentences=pd.read_csv(pathHelper2)
unarxiveSentences=pd.read_csv(unarxiveTestingInput)
df.head()

,Sentence,Section
0,Introduction Real-world usage of IoT in health...,intro
1,"Indeed, the discovery of vulnerabilities of si...",intro
2,"In fact, various ones have been defined in the...",intro
3,Its wide study led to the birth of several def...,intro
4,"In this paper, we wish to bridge the gap betwe...",intro


Verschiedene Sections finden


In [ ]:
print(df)

                                                Sentence     Section
0      Introduction Real-world usage of IoT in health...       intro
1      Indeed, the discovery of vulnerabilities of si...       intro
2      In fact, various ones have been defined in the...       intro
3      Its wide study led to the birth of several def...       intro
4      In this paper, we wish to bridge the gap betwe...       intro
...                                                  ...         ...
12496  Conclusions The alien character has been subje...  discussion
12497  Each look is a moment in which the text return...  discussion
12498  Isolation's auto-critique of gaming's technici...  discussion
12499  The centrality of ego-centric design imperativ...  discussion
12500  Isolation's commitment to a look of psychopath...  discussion

[12501 rows x 2 columns]


In [ ]:
label_dict = {}
label_dict["intro"]=0
label_dict["related"]=1
label_dict["method"]=2
label_dict["results"]=3
label_dict["discussion"]=4
print(label_dict)

{'intro': 0, 'related': 1, 'method': 2, 'results': 3, 'discussion': 4}


In [ ]:
df = df[df['Section'].notna()]
testSentences=testSentences[testSentences['Section'].notna()]

In [ ]:
def sampleDataframe(size,df):
  intro = df.loc[df.Section == 'intro']
  related = df.loc[df.Section == 'related']
  method = df.loc[df.Section == 'method',]
  results = df.loc[df.Section == 'results']
  discussion = df.loc[df.Section == 'discussion',]
 
  result = pd.concat([intro.sample(n=size, replace=False).reset_index(drop=True), 
                    related.sample(n=size, replace=False).reset_index(drop=True),
                    method.sample(n=size, replace=False).reset_index(drop=True),
                    results.sample(n=size, replace=False).reset_index(drop=True),
                    discussion.sample(n=size, replace=False).reset_index(drop=True)], axis=0)
  
  return result

In [ ]:
df=sampleDataframe(1000,df)


In [ ]:
testSentences=sampleDataframe(250,testSentences)

In [ ]:
df['Section'].value_counts()
possible_labels = df.Section.unique()

df['label'] = df.Section.replace(label_dict)


testSentences['Section'].value_counts()
testSentences['label'] = testSentences.Section.replace(label_dict)
unarxiveSentences['label'] = unarxiveSentences.Section.replace(label_dict)

testSentences.head()
#print(label_dict)
print(possible_labels)
print(unarxiveSentences)

['intro' 'related' 'method' 'results' 'discussion']
                                              Sentence     Section  label
0    Introduction Some of the most relevant global ...       intro      0
1    Usually, it is considered as the gold standard...       intro      0
2    The main goal of this work is to provide an ac...       intro      0
3    Which in our case are the myocardial tissue an...       intro      0
4    To overcome this problem the technique of batc...       intro      0
..                                                 ...         ...    ...
598  It is empirically found that a control group o...  discussion      4
599  When we compare our characteristics like popul...  discussion      4
600  Discussion Efficient stochastic approximation ...  discussion      4
601     Discussion In this paper, we introduced the...  discussion      4
602  Furthermore, on networks with background nodes...  discussion      4

[603 rows x 3 columns]


In Test und Validationsdatensatz aufteilen
Intro Label=0;Methods 1, results 2, Discussion 3

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(df.index.values,df.label.values,test_size=0.03,random_state=42, stratify=df.label.values)

df['data_type'] = ['not_set']*df.shape[0]

df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

print(df.groupby(['Section', 'label', 'data_type']).count())
print(testSentences.groupby(['Section', 'label',]).count())
print(unarxiveSentences.groupby(['Section', 'label',]).count())


                            Sentence
Section    label data_type          
discussion 4     train           862
                 val             138
intro      0     train           862
                 val             138
method     2     train           862
                 val             138
related    1     train           862
                 val             138
results    3     train           862
                 val             138
                  Sentence
Section    label          
discussion 4           250
intro      0           250
method     2           250
related    1           250
results    3           250
                  Sentence
Section    label          
discussion 4            81
intro      0            87
method     2           215
related    1            84
results    3           136


Gesamten Batch encoden. Aufpassen: Tokenizer muss der gleiche wie das Model sein, damit das Vokabular übereinstimmt. --> Später Scibert

allenai/scibert_scivocab_cased


In [ ]:
array1=(df[df.data_type=='train'].Sentence.values).tolist()
for i in range (0,len(array1)):
  array1[i]=str(array1[i])

array2=(df[df.data_type=='val'].Sentence.values).tolist()
for i in range (0,len(array2)):
  array2[i]=str(array2[i])


array3=(testSentences.Sentence.values).tolist()
for i in range (0,len(array3)):
  array3[i]=str(array3[i])

array4=(unarxiveSentences.Sentence.values).tolist()
for i in range (0,len(array4)):
  array4[i]=str(array4[i])


In [ ]:
tokenizer = BertTokenizer.from_pretrained('allenai/scibert_scivocab_cased',do_lower_case=True)

encoded_data_train = tokenizer.batch_encode_plus(
    array1, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    array2, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)
encoded_data_test= tokenizer.batch_encode_plus(
    array3, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)
encoded_data_unarxive= tokenizer.batch_encode_plus(
    array4, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

input_ids_test = encoded_data_test['input_ids']
attention_masks_test = encoded_data_test['attention_mask']
labels_test = torch.tensor(testSentences.label.values)

dataset_test = TensorDataset(input_ids_test, attention_masks_test, labels_test)

input_ids_unarxive = encoded_data_unarxive['input_ids']
attention_masks_unarxive = encoded_data_unarxive['attention_mask']
labels_unarxive = torch.tensor(unarxiveSentences.label.values)

dataset_unarxive = TensorDataset(input_ids_unarxive, attention_masks_unarxive, labels_unarxive)



Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2190: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Model festlegem --> später auch Scibert

In [ ]:
model = BertForSequenceClassification.from_pretrained("allenai/scibert_scivocab_cased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False, #allenai/scibert_scivocab_cased
                                                      output_hidden_states=False)

Some weights of the model checkpoint at allenai/scibert_scivocab_cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were

Daten Laden und zufällig Mischen

In [ ]:

batch_size = 10

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

dataloader_test = DataLoader(dataset_test, 
                                   sampler=SequentialSampler(dataset_test), 
                                   batch_size=batch_size)

dataloader_unarxive = DataLoader(dataset_unarxive, 
                                   sampler=SequentialSampler(dataset_unarxive), 
                                   batch_size=batch_size)


Schrittweite mit AdamW festlegen

In [ ]:

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)
                  
epochs = 5

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

F1 Score als Bewertungsgrundlage importieren

In [ ]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

Über Cuda Grafikkarte ansteuren und
Modell 5 Epochen trainieren

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = "cuda:0"
model = model.to(device)
print(device)
counterSave=0
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

def evaluate(dataloader_val):
    
    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []


    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals
    
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    counterSave=counterSave+1
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
    if(counterSave>3):
      torch.save(model.state_dict(), f'/content/drive/MyDrive/Seminar_KD_aifb/Scibert_Unpaywall/UnpaywallFinetuned_NORMALBERT002_epoch_100words_batches_bigNEW{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

cuda:0



Epoch 1
Training loss: 0.8789004426613761
Validation loss: 0.7084321909840556
F1 Score (Weighted): 0.7509105889954821



Epoch 2
Training loss: 0.5351477656038067
Validation loss: 0.7259044531678808
F1 Score (Weighted): 0.7401301252848504



Epoch 3
Training loss: 0.3737456723701926
Validation loss: 0.7407880770987358
F1 Score (Weighted): 0.7591055130455303



Epoch 4
Training loss: 0.26132446214591265
Validation loss: 0.7951858237763678
F1 Score (Weighted): 0.7718613738275844



Epoch 5
Training loss: 0.18453836794337583
Validation loss: 0.8446538716662622
F1 Score (Weighted): 0.7849723528217666



Nur Test für Cuda, nicht weiter Relevant.

In [ ]:
device = torch.device('cuda')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

Auswertung durchführen

In [ ]:
model = BertForSequenceClassification.from_pretrained("allenai/scibert_scivocab_cased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)
tokenizer = BertTokenizer.from_pretrained('allenai/scibert_scivocab_cased',do_lower_case=True)

Some weights of the model checkpoint at allenai/scibert_scivocab_cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were

In [ ]:


model.to(device)

model.load_state_dict(torch.load('/content/drive/MyDrive/Seminar_KD_aifb/Scibert_Unpaywall/UnpaywallFinetuned_NORMALBERT002_epoch_100words_batches_bigNEW4.model', map_location=torch.device('cpu')))
"""/content/drive/MyDrive/Seminar_KD_aifb/Scibert_Unpaywall/UnpaywallFinetuned_SciBERT002_epoch_100words_batches5.model"""
_, predictions, true_vals = evaluate(dataloader_validation)
accuracy_per_class(predictions, true_vals)

Class: intro
Accuracy: 106/138

Class: related
Accuracy: 122/138

Class: method
Accuracy: 101/138

Class: results
Accuracy: 105/138

Class: discussion
Accuracy: 99/138



In [ ]:
def toDataloader(data):
  label_dict = {}
  label_dict["intro"]=0
  label_dict["related"]=1
  label_dict["method"]=2
  label_dict["results"]=3
  label_dict["discussion"]=4
  print(data['Section'].value_counts())

  data['label'] = data.Section.replace(label_dict)
  (data.groupby(['Section', 'label',]).count())
  array5=(data.Sentence.values).tolist()
  print(len(array5))
  for i in range (0,len(array5)):
    array5[i]=str(array5[i])

  encodedFrame= tokenizer.batch_encode_plus(
      array5, 
      add_special_tokens=True, 
      return_attention_mask=True, 
      pad_to_max_length=True, 
      max_length=256, 
      return_tensors='pt'
  )
  input_ids_Frame = encodedFrame['input_ids']
  attention_masks_Frame = encodedFrame['attention_mask']
  labels_Frame = torch.tensor(data.label.values)

  dataset_frame = TensorDataset(input_ids_Frame, attention_masks_Frame, labels_Frame)
  dataloader_test = DataLoader(dataset_frame, 
                                    sampler=SequentialSampler(dataset_frame), 
                                    batch_size=batch_size)
  return dataloader_test

In [ ]:
data = pd.read_csv(pathHelper2)
data=sampleDataframe(250,data)
#type(data)
x=toDataloader(data)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


discussion    250
method        250
results       250
intro         250
related       250
Name: Section, dtype: int64
1250


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2132: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
testsss = pd.read_csv(unarxiveTestingInput)

zz=sampleDataframe(50,testsss)
yy=toDataloader(zz)


results       50
discussion    50
related       50
method        50
intro         50
Name: Section, dtype: int64
250


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2132: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
_, predictions, true_vals = evaluate(yy)
accuracy_per_class(predictions, true_vals)
val_f1 = f1_score_func(predictions, true_vals)
tqdm.write(f'F1 Score (Weighted): {val_f1}')

Class: intro
Accuracy: 37/50

Class: related
Accuracy: 30/50

Class: method
Accuracy: 40/50

Class: results
Accuracy: 33/50

Class: discussion
Accuracy: 32/50

F1 Score (Weighted): 0.6868716494987326


In [ ]:
testSentences=pd.read_csv(pathHelper2)

ll=sampleDataframe(250,testSentences)
bb=toDataloader(ll)

_, predictions, true_vals = evaluate(bb)
accuracy_per_class(predictions, true_vals)
val_f1 = f1_score_func(predictions, true_vals)
tqdm.write(f'F1 Score (Weighted): {val_f1}')

discussion    250
method        250
results       250
intro         250
related       250
Name: Section, dtype: int64
1250


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2132: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Class: intro
Accuracy: 184/250

Class: related
Accuracy: 211/250

Class: method
Accuracy: 184/250

Class: results
Accuracy: 200/250

Class: discussion
Accuracy: 197/250

F1 Score (Weighted): 0.7810570060578208


In [ ]:
_, predictions, true_vals = evaluate(dataloader_unarxive)
accuracy_per_class(predictions, true_vals)


Class: intro
Accuracy: 65/87

Class: related
Accuracy: 42/84

Class: method
Accuracy: 162/215

Class: results
Accuracy: 81/136

Class: discussion
Accuracy: 52/81



In [ ]:
from transformers import BertForSequenceClassification, BertTokenizer, TextClassificationPipeline

In [ ]:
model = BertForSequenceClassification.from_pretrained("allenai/scibert_scivocab_cased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.load_state_dict(torch.load('/content/drive/MyDrive/Seminar_KD_aifb/Scibert_Unpaywall/UnpaywallFinetuned_SciBERT001_epoch_200words5.model', map_location=torch.device('cuda')))
pipeline=TextClassificationPipeline(model=model, tokenizer = BertTokenizer.from_pretrained('allenai/scibert_scivocab_cased'),device=0)


In [ ]:
print(device)

In [ ]:
df = pd.read_csv(pathHelper2)
sentencesFrame=df['Sentence']
labelFrame=df['Section']

In [ ]:
print(labelFrame[1])

In [ ]:
pipeline("methods")

In [ ]:
listWithLabels=["intro","related","method","results","discussion"]
listWithOtherLabels=["LABEL_0","LABEL_1","LABEL_2","LABEL_3","LABEL_4"]

z=0
k=0
resultAsDiscussion=0
DiscussionAsResult=0
for i in tqdm(sentencesFrame):
  a=pipeline(i)
  b=labelFrame[z]
  if b=="intro":
    if a[0]["label"]!="LABEL_0":
      #print (f"Labeles as {a[0]} but is {b}: {i} ")
      k=k+1

  if b=="related":
    if a[0]["label"]!="LABEL_1":
      #print (f"Labeles as {a[0]} but is {b}: {i} ")
      k=k+1

  if b=="method":
    if a[0]["label"]!="LABEL_2":
      #print (f"Labeles as {a[0]} but is {b}: {i} ")
      k=k+1

  if b=="results":
    if a[0]["label"]!="LABEL_3":
      #print (f"Labeles as {a[0]} but is {b}: {i} ")
      k=k+1
      if a[0]["label"]=="LABEL_4":
        DiscussionAsResult+=1

  if b=="discussion":
    if a[0]["label"]!="LABEL_4":
      #print (f"Labeles as {a[0]} but is {b}: {i} ")
      k=k+1
      if a[0]["label"]=="LABEL_3":
        resultAsDiscussion+=1

  z=z+1
print(k)
print(resultAsDiscussion)
print(DiscussionAsResult)

RuntimeError: ignored

In [ ]:
print(k)
print(resultAsDiscussion)
print(DiscussionAsResult)